In [1]:
import pandas as pd

In [2]:
CITY = "budapest"

In [12]:
distance = pd.read_csv(f"../output/{CITY}/distance.csv")
ac = pd.read_csv(f"../output/{CITY}/amenity_counts_in_accessibility.csv")
pt_ac = pd.read_csv(
    f"../output/{CITY}/amenity_counts_in_public_transport_accessibility.csv"
)
stop_geometries = pd.read_csv(f"../output/{CITY}/stop_geometries_from_walk.csv")

stop_centralities = pd.read_csv(f"../data/stops/{CITY}/stops_with_centrality.csv")
stop_centralities.drop(["Node"], axis=1, inplace=True)
stop_centralities.columns = [
    "eigenvector_centrality",
    "degree_centrality",
    "closeness_centrality",
    "betweenness_centrality",
    "stop_id",
    "cluster",
    "stop_lat",
    "stop_lon",
    "stop_name",
]
stop_centralities.dropna(subset=["stop_id"], inplace=True)

In [13]:
stop_centralities

,eigenvector_centrality,degree_centrality,closeness_centrality,betweenness_centrality,stop_id,cluster,stop_lat,stop_lon,stop_name
0,3.632075e-15,0.000437,0.033463,0.000000,044369,1.0,47.175605,18.941796,Ráckeve
1,1.878265e-14,0.001311,0.034621,0.002620,CSF04632,2.0,47.227924,18.946067,Szigetszentmárton-Szigetújfalu
3,8.972862e-14,0.001311,0.035859,0.003490,009749,6.0,47.270708,18.956638,Szigetcsép H
4,8.972862e-14,0.001311,0.035859,0.003490,CSF04627,6.0,47.270231,18.956263,Szigetcsép
6,8.668510e-14,0.000874,0.035857,0.000000,008809,5.0,47.264682,18.970417,"Szigetcsép, községháza"
...,...,...,...,...,...,...,...,...,...
5014,8.414154e-05,0.001311,0.055587,0.010352,F04430,2438.0,47.428510,19.007951,Liszt Ferenc út
5015,1.704007e-05,0.000874,0.052690,0.005085,009460,2445.0,47.424593,19.010866,Komáromi út
5016,1.704007e-05,0.000874,0.052690,0.005085,009461,2445.0,47.423955,19.011131,Komáromi út
5017,5.459805e-04,0.001311,0.062184,0.030686,009546,2442.0,47.425665,19.001313,Memento Park


In [14]:
wk_amenity = ac.query("costing == 'walk' & range == 15").copy()
wk_amenity.drop(["costing", "range"], axis=1, inplace=True)
wk_amenity.columns = ["stop_id"] + [f"{i}_walk15" for i in wk_amenity.columns[1:]]
mm_amenity = pt_ac.copy()
mm_amenity.drop(["costing", "range"], axis=1, inplace=True)
mm_amenity.columns = ["stop_id"] + [f"{i}_multimodal" for i in mm_amenity.columns[1:]]

In [15]:
m = (
    stop_geometries.drop("geometry", axis=1)
    .merge(distance, on="stop_id")
    .merge(mm_amenity, on="stop_id")
    .merge(wk_amenity, on="stop_id")
    .merge(stop_centralities, on="stop_id")
)
m.to_csv(f"../output/{CITY}/merged.csv", index=False)